In [71]:

import os
import requests
import sqlite3
import pickle
from datetime import datetime, timedelta
from datetime import timezone
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from dotenv import load_dotenv
from IPython.display import clear_output
import util
from importlib import reload
load_dotenv()
reload(util)
COINGECKO_API = os.getenv("COINGECKO_API_KEY")
HEADERS = {"x-cg-pro-api-key": COINGECKO_API,
           "accept": "application/json"}

In [73]:
layer1_coins = util.get_coins_from_category("layer-1")

In [4]:
conn = sqlite3.connect("../Data/crypto_data.db")
cursor = conn.cursor()

# Create table for coin history with OHLC and market data
cursor.execute("""
CREATE TABLE IF NOT EXISTS coin_history (
    coin_id TEXT,
    symbol TEXT,
    name TEXT,
    time_rank TEXT,
    open REAL,
    high REAL,
    low REAL,
    close REAL,
    volume REAL,
    market_cap REAL,
    PRIMARY KEY (coin_id, time_rank)
)
""")
conn.commit()

In [5]:
def fetch_ohlc_chart_data(coin_id, from_ts, to_ts, vs_currency="usd"):
    """
    Use the Coin OHLC Chart Data endpoint.
    Returns a list of [timestamp, open, high, low, close] arrays.
    Timestamps (in ms) refer to the end of each interval.
    This endpoint supports up to 31 days at a time.
    """
    url = f"https://pro-api.coingecko.com/api/v3/coins/{coin_id}/ohlc/range"
    params = {
        "vs_currency": vs_currency,
        "from": int(from_ts),
        "to": int(to_ts),
        "interval": "hourly"
    }
    resp = requests.get(url, params=params, headers=HEADERS)
    resp.raise_for_status()
    return resp.json()
def fetch_historical_chart_data(coin_id, from_ts, to_ts, vs_currency="usd"):
    """
    Use the Coin Historical Chart Data endpoint.
    Returns a dict with keys 'prices', 'market_caps', and 'total_volumes'.
    Timestamps in the returned lists are in milliseconds.
    This endpoint now supports up to 100 days of hourly data.
    """
    url = f"https://pro-api.coingecko.com/api/v3/coins/{coin_id}/market_chart/range"
    params = {
        "vs_currency": vs_currency,
        "from": int(from_ts),
        "to": int(to_ts),
    }
    resp = requests.get(url, params=params, headers=HEADERS)
    resp.raise_for_status()
    return resp.json()

In [74]:
def ceil_to_hour(dt: datetime) -> datetime:
    """Return the datetime rounded down to the nearest hour."""
    return dt + timedelta(minutes=60 - dt.minute, seconds=-dt.second, microseconds=-dt.microsecond)

def round_up_timestamp_to_nearest_hour(timestamp):
    return math.ceil(timestamp / 3600) * 3600

def update_coin_history(coin, start_date="2018-02-01"):
    """
    For a given coin (dict with id, symbol, name), update the coin_history table.
    Data is fetched in 31-day chunks for OHLC (as it supports only 31 days at a time).
    Before inserting, the timestamp from OHLC is floored to the hour mark.
    """
    coin_id = coin["id"]
    symbol = coin["symbol"]
    name = coin["name"]
    
    # Get the latest timestamp stored for this coin (if any)
    cursor.execute("SELECT MAX(time_rank) FROM coin_history WHERE coin_id=?", (coin_id,))
    result = cursor.fetchone()
    if result[0]:
        latest = datetime.fromisoformat(result[0])
        start_dt = latest + timedelta(seconds=1)
    else:
        start_dt = datetime.strptime(start_date, "%Y-%m-%d")
    print(f"Latest data for {coin_id} is available until {start_dt}")
    start_dt = start_dt.replace(tzinfo=timezone.utc)
    
    now = datetime.now(timezone.utc)
    if start_dt >= now:
        return  # Data is up-to-date
    
    print(f"Updating {coin_id} data from {start_dt.isoformat()} to {ceil_to_hour(now-pd.Timedelta(hours = 1)).isoformat()}")

    # Use 31-day chunks as per OHLC endpoint limits.
    chunk = timedelta(days=31)
    curr_start = start_dt
    while curr_start < now:
        curr_end = min(ceil_to_hour(now-pd.Timedelta(hours =1)), curr_start + chunk)
        if curr_end < curr_start+timedelta(hours=1):
            break
        clear_output(wait=True)
        print(f"Fetching data for {coin_id} from {curr_start.isoformat()} to {curr_end.isoformat()} ")
        from_ts = curr_start.timestamp()  # seconds
        to_ts = min(curr_end.timestamp(), now.timestamp())        # seconds
        
        try:
            # Fetch historical chart data (prices, market_caps, volumes)
            
            hist_data = fetch_historical_chart_data(coin_id, from_ts-3600, to_ts)
            # Build lookup dictionaries using timestamp in ms
            price_dict = {round_up_timestamp_to_nearest_hour(int(item[0]/1000)): item[1] for item in hist_data.get("prices", [])}
            mcap_dict = {round_up_timestamp_to_nearest_hour(int(item[0]/1000)): item[1] for item in hist_data.get("market_caps", [])}
            vol_dict = {round_up_timestamp_to_nearest_hour(int(item[0]/1000)): item[1] for item in hist_data.get("total_volumes", [])}
            
            # Fetch OHLC chart data for the same range.
            ohlc_data = fetch_ohlc_chart_data(coin_id, from_ts, to_ts)
            # if not ohlc_data:
            #     print(f"No OHLC data for {coin_id} in this chunk.")
            #     break
            # Process each OHLC data point.
            # Note: The timestamp from OHLC refers to the end of the interval.
            for entry in ohlc_data:
                ts_ms = int(entry[0])/1000
                # Convert timestamp to datetime and floor to the hour
                dt = datetime.fromtimestamp(ts_ms, tz=timezone.utc)
                dt_hour = ceil_to_hour(dt)
                dt_iso = dt_hour.isoformat()
                
                open_price, high_price, low_price, close_price = entry[1], entry[2], entry[3], entry[4]
                volume = vol_dict.get(ts_ms, None)
                market_cap = mcap_dict.get(ts_ms, None)
                
                # Insert data into the database; duplicates (same coin_id and time_rank) are skipped.
                cursor.execute("""
                    INSERT OR IGNORE INTO coin_history (
                        coin_id, symbol, name, time_rank, open, high, low, close, volume, market_cap
                    ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
                """, (coin_id, symbol, name, dt_iso, open_price, high_price, low_price, close_price, volume, market_cap))
            conn.commit()
            print(f"Inserted {len(ohlc_data)} records for {coin_id} in this chunk.")
        except Exception as e:
            print(f"Error updating {coin_id} for chunk {curr_start.isoformat()} to {curr_end.isoformat()}: {e}")
        curr_start = curr_end + timedelta(seconds=1)
        # if len(ohlc_data) < 700:
        #     break

In [63]:
for coins in layer1_coins:
    update_coin_history(coins)

Fetching data for shardeum from 2018-02-01T00:00:00+00:00 to 2018-03-04T00:00:00+00:00 
Inserted 0 records for shardeum in this chunk.


In [69]:
reload(util)
gecko_coins = util.get_all_coin_names()
coindict = {coin['id']: coin for coin in gecko_coins}
kraken_coin_data = util.get_exchange_coins('kraken')
kraken_coin_ids = [coindict[coin['coin_id']] for coin in kraken_coin_data]
count = 1
for coin in kraken_coin_ids:
    print(f"Updating {count}/{len(kraken_coin_ids)}: {coin['id']}")
    count += 1
    update_coin_history(coin)

Fetching data for maker from 2025-03-08T17:00:01+00:00 to 2025-03-08T19:00:00+00:00 
Inserted 3 records for maker in this chunk.
Updating 57/98: pudgy-penguins
Latest data for pudgy-penguins is available until 2025-03-08 18:00:01+00:00
Updating pudgy-penguins data from 2025-03-08T18:00:01+00:00 to 2025-03-08T19:00:00+00:00
Updating 58/98: jupiter-exchange-solana
Latest data for jupiter-exchange-solana is available until 2025-03-08 18:00:01+00:00
Updating jupiter-exchange-solana data from 2025-03-08T18:00:01+00:00 to 2025-03-08T19:00:00+00:00
Updating 59/98: ai16z
Latest data for ai16z is available until 2025-03-08 18:00:01+00:00
Updating ai16z data from 2025-03-08T18:00:01+00:00 to 2025-03-08T19:00:00+00:00
Updating 60/98: griffain
Latest data for griffain is available until 2025-03-08 18:00:01+00:00
Updating griffain data from 2025-03-08T18:00:01+00:00 to 2025-03-08T19:00:00+00:00
Updating 61/98: fwog
Latest data for fwog is available until 2025-03-08 18:00:01+00:00
Updating fwog data

In [18]:
query = "SELECT time_rank, coin_id, symbol, name, open, high, low, close, volume, market_cap FROM coin_history"
df = pd.read_sql_query(query, conn)
df['time_rank'] = pd.to_datetime(df['time_rank'])
print(df.head())

                  time_rank  coin_id symbol     name     open     high  \
0 2018-02-02 02:00:00+00:00  bitcoin    btc  Bitcoin  9070.56  9070.56   
1 2018-02-03 02:00:00+00:00  bitcoin    btc  Bitcoin  8830.16  8830.16   
2 2018-02-04 02:00:00+00:00  bitcoin    btc  Bitcoin  8994.91  8994.91   
3 2018-02-05 02:00:00+00:00  bitcoin    btc  Bitcoin  8315.52  8315.52   
4 2018-02-06 02:00:00+00:00  bitcoin    btc  Bitcoin  6852.05  6852.05   

       low    close        volume    market_cap  
0  9070.56  9070.56  4.720685e+09  1.490066e+11  
1  8830.16  8830.16  6.689345e+09  1.470204e+11  
2  8994.91  8994.91  3.406201e+09  1.529524e+11  
3  8315.52  8315.52  3.313033e+09  1.377500e+11  
4  6852.05  6852.05  5.582367e+09  1.166751e+11  
